In [1]:
import warnings
warnings.filterwarnings("ignore")

from keras.models import model_from_json
import numpy as np


EMOTIONS_LIST = ["Angry", "Disgust", "Fear", "Happy","Neutral", "Sad", "Surprise"]

model_json_file ="./model3.json"
model_weights_file = "./model_weights5.h5"
with open(model_json_file, "r") as json_file:
    loaded_model_json = json_file.read()
    loaded_model = model_from_json(loaded_model_json)

# load weights into the new model
loaded_model.load_weights(model_weights_file)
#loaded_model._make_predict_function()

Using TensorFlow backend.


### 1. 예측 haar 실습

In [2]:
import cv2
import sys

font = cv2.FONT_HERSHEY_SIMPLEX
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

video_capture = cv2.VideoCapture(0)

# img_counter = 0

while video_capture.isOpened:

    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    k = cv2.waitKey(33)
    faces = faceCascade.detectMultiScale(
        gray, scaleFactor=1.5, minNeighbors=5,
        #         minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    # 획득 이미지 회색컬러로 변경
    gray_fr = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # 인식된 얼굴 주위에 사각형 영역 표시
    # haar cascading 통해 face 이미지 추출 후 좌표활용)
    for (x, y, w, h) in faces:
        fc = gray_fr[y:y+h, x:x+w]

        # 획득 이미지 크기조절 및 딥러닝 입력 형태로 변환
        roi = cv2.resize(fc, (48, 48))
        roip = roi.reshape(1,48,48,1)
             
        # 예측
        predict = loaded_model.predict(roip)
        # 예측결과 리턴
        pred = EMOTIONS_LIST[np.argmax(predict)]
        # 예측결과 화면 시연 및 사각형 영역 표시
        cv2.putText(frame, pred, (x, y), font, 1, (255, 255, 0), 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # 화면 출력
    cv2.imshow('FaceDetection', frame)

    if k == 27: #ESC Pressed
        break
#     elif k == 32:
#         # SPACE pressed
#         img_name = "facedetect_webcam.png"
#         cv2.imwrite(img_name, frame)
        

# 종료 후 캡쳐 릴리즈 및 윈도우 종료
video_capture.release()
cv2.destroyAllWindows()

### 2. 일반 haar 실습

In [ ]:
import cv2
import sys

faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

video_capture = cv2.VideoCapture(0)

# img_counter = 0

while video_capture.isOpened:

    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    k = cv2.waitKey(33)
    faces = faceCascade.detectMultiScale(
        gray, scaleFactor=1.5, minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    # 인식된 얼굴 주위에 사각형 영역 표시
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # 화면 출력
    cv2.imshow('FaceDetection', frame)

    if k == 27: #ESC Pressed
        break
    elif k == 32:
        # SPACE pressed
        img_name = "facedetect_webcam.png"
        cv2.imwrite(img_name, frame)
        img_counter += 1
        

# 종료 후 캡쳐 릴리즈 및 윈도우 종료
video_capture.release()
cv2.destroyAllWindows()